TODO:

- [ ] Mining metaculus (and other markets) for Numerai signals. This is especially interesting giving the trade signal tournament https://www.metaculus.com/tournament/trade-signal-tournament/ and  the community trader election 
https://www.metaculus.com/questions/7330/community-trader-election/
- [ ] [Someone on r/datascience](https://www.reddit.com/r/datascience/comments/nmiq15/i_used_vader_sentiment_analysis_to_track_and/) used VADER sentiment analysis to invest based on WallStreetBets sentiment. See [code](https://github.com/tstewart161/Reddit_Sentiment_Trader) and [explanatory article](https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664).
- [ ] Build a minimum viable submission with `ticker` and `signal` columns for validation data and submit for feedback.
- [ ] "Check out [this forum thread](https://forum.numer.ai/t/free-or-cheap-data-for-erasure-numerai-quant/350) for a list of sources popular data sources, platforms, and tools used by our community."
- [ ] "there are a number of free or cheap data providers on the internet such as Yahoo Finance, Quandl, and Koyfin."
- [ ] "There are also platforms that make it easy to create signals such as QuantConnect, and Alpaca."
- [x] Load and explore [universe file](https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/universe/latest.csv)
- [x] Load and explore [historical targets file](https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_train_val_bbg.csv)

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
from pathlib import Path
DIR = Path('gdrive/MyDrive/numerai')
SIGDIR = DIR / 'signals'
DATADIR = DIR / 'data'
SRCDIR = DIR / 'src'
RESULTDIR = DIR / 'results'

In [ ]:
# Copy .env from numerai folder to root dir
!cp gdrive/MyDrive/Data/numerai/.env .env

In [ ]:
%%capture
# install
# !pip uninstall --no-input pandas
!pip install --upgrade python-dotenv fastai numerapi
!pip install ipython-autotime
!pip install investpy
!pip install yfinance --upgrade --no-cache-dir
!pip install pycountry
# a fork of yfinance that implements retries nicely
# !pip install -e git+http://github.com/leonhma/yfinance.git@master#egg=yfinance

# import dependencies
import gc
import os
import csv
from dotenv import load_dotenv, find_dotenv
from getpass import getpass
import numerapi
from scipy.stats import spearmanr
import sklearn.linear_model
import investpy
import yfinance
import datetime

from tqdm import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
SIGNALPATH = SIGDIR / '269'

df = pd.read_csv(SIGNALPATH / '2021-06-19_Core Portfolio.csv')
dfx = pd.read_json(SIGNALPATH/ '1624143570_list.json').T
df = pd.merge(df, dfx, left_on='Nosible UUID', right_on='nosible_uuid', how='left')
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Columns: 109 entries, Unnamed: 0 to overall_fit
dtypes: float64(38), object(71)
memory usage: 101.4+ KB


In [ ]:
from sklearn.preprocessing import MinMaxScaler

_df = pd.DataFrame()
_df['numerai_ticker'] = df['bloomberg_code'].apply(lambda s: ' '.join(str(s).split()[:-1]) if s is not None else '')
_df['data_type'] = 'live'
_df['friday_date'] = 20210618
_df['signal'] = MinMaxScaler(feature_range=(0.1, 0.9)).fit_transform(df['Quality'].values.reshape(-1, 1))

_df = _df[_df.numerai_ticker != '']
print(_df.shape)
_df.to_csv(SIGNALPATH/'NO_SIBYL_269.csv', index=False)
_df.sample(10)

(100, 4)


,numerai_ticker,data_type,friday_date,signal
64,BSX US,live,20210618,0.573469
42,ADVM US,live,20210618,0.393878
100,GRAY US,live,20210618,0.370748
69,MBRX US,live,20210618,0.393878
74,TECH US,live,20210618,0.822449
109,ALGS US,live,20210618,0.370748
26,ADPT US,live,20210618,0.425170
29,XOMA US,live,20210618,0.498639
32,ALXN US,live,20210618,0.720408
93,PBYI US,live,20210618,0.350340


In [ ]:
dfx = pd.read_json(SIGDIR/'269'/'1624143570_list.json').T
_df = pd.merge(df, dfx, left_on='Nosible UUID', right_on='nosible_uuid', how='left')
_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 0 to 117
Columns: 109 entries, Unnamed: 0 to overall_fit
dtypes: float64(38), object(71)
memory usage: 101.4+ KB


In [ ]:
!pip install fuzzywuzzy

In [ ]:
df_lookup = pd.DataFrame(
    investpy.stocks.get_stocks_dict(
        country=None,
        columns=["symbol", "country", "name", "full_name", "isin", "currency"],
        as_json=False,
    )
)
df_lookup.sample(10)

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,symbol,country,name,full_name,isin,currency
12704,PLMP,poland,Polman SA,Polman SA,PLPOLMN00010,PLN
30711,VITA,philippines,Vitarich,Vitarich Corp,PHY937931186,PHP
9697,SMCP,france,Smcp,Smcp SAS,FR0013214145,EUR
20112,002547,china,Chunxing Pre Mec A,SuZhou ChunXing Precision Mechanical Co Ltd,CNE100001047,CNY
29346,MHCP,malaysia,MHC Plantations,MHC Plantations Bhd,MYL5026OO009,MYR
17491,AOK,australia,AusTex Oil Ltd,AusTex Oil Ltd,AU000000AOK3,AUD
20858,300442,china,Shanghai Precise Packaging,Shanghai Precise Packaging Co Ltd,CNE100001Z58,CNY
16846,RDSb,united kingdom,Royal Dutch Shell B,Royal Dutch Shell PLC B,GB00B03MM408,GBP
11924,SSOL,norway,Scatec Solar OL,Scatec Solar OL,NO0010715139,NOK
23337,BRTI,india,Bharti Airtel,Bharti Airtel Ltd,INE397D01024,INR


In [ ]:
df_lookup[df_lookup['isin'] == 'US6700024010']

,symbol,country,name,full_name,isin,currency
4696,NVAX,united states,Novavax,Novavax Inc,US6700024010,USD


In [ ]:
from fuzzywuzzy import fuzz, process

for c in df['Name'].unique():
    print(c, process.extract(c, df_lookup['full_name'].to_list(), limit=3))

Ceres Inc [('Ceres Inc', 100), ('Caleres Inc', 90), ('INC', 90)]
Canopy Rivers Inc [('INC', 90), ('Verizon Communications Inc DRC', 86), ('HP Inc DRC', 86)]
Terns Pharmaceuticals Inc [('Xeris Pharmaceuticals Inc', 92), ('Supernus Pharmaceuticals Inc', 91), ('Ra Pharmaceuticals Inc', 89)]
Satsuma Pharmaceuticals Inc Or... [('Ra Pharmaceuticals Inc', 88), ('Citigroup Inc DRC', 86), ('AT&T Inc DRC', 86)]
Vaxcell-Bio Therapeutics Ordin... [('Hera', 90), ('HLS Therapeutics Inc', 86), ('iCo Therapeutics Inc', 86)]


KeyboardInterrupt: ignored

## Modifying the Numerai example

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, FR
from sklearn.ensemble import GradientBoostingRegressor

def RSI(prices, interval=10):
    '''Computes Relative Strength Index given a price series and lookback interval
  Modified from https://stackoverflow.com/questions/20526414/relative-strength-index-in-python-pandas
  See more here https://www.investopedia.com/terms/r/rsi.asp'''
    delta = prices.diff()

    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(interval).mean()
    RolDown = dDown.rolling(interval).mean().abs()

    RS = RolUp / RolDown
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI

In [ ]:
'''Creates example_signal_upload.csv to upload for validation and live data submission'''
napi = numerapi.SignalsAPI()

# read in list of active Signals tickers which can change slightly era to era
eligible_tickers = pd.Series(napi.ticker_universe(), name='numerai_ticker')
print(f"Number of eligible tickers: {len(eligible_tickers)}")

# read in yahoo to numerai ticker map, still a work in progress, h/t wsouza
ticker_map = pd.read_csv(
    'https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_ticker_map_w_bbg.csv'
)
print(f"Number of tickers in map: {len(ticker_map)}")

# map eligible numerai tickers to yahoo finance tickers
yfinance_tickers = eligible_tickers.map(
    dict(zip(ticker_map['bloomberg_ticker'], ticker_map['yahoo']))).dropna()
numerai_tickers = ticker_map['bloomberg_ticker']
print(f'Number of eligible, mapped tickers: {len(yfinance_tickers)}')

# download data
n = 1000  # chunk row size
chunk_df = [
    yfinance_tickers.iloc[i:i + n]
    for i in range(0, len(yfinance_tickers), n)
]

concat_dfs = []
print("Downloading data...")
for df in chunk_df:
    try:
        # set threads = True for faster performance, but tickers will fail, script may hang
        # set threads = False for slower performance, but more tickers will succeed
        temp_df = yfinance.download(df.str.cat(sep=' '),
                                    start='2019-01-01',
                                    threads=False)
        temp_df = temp_df['Adj Close'].stack().reset_index()
        concat_dfs.append(temp_df)
    except:  # simplejson.errors.JSONDecodeError:
        pass

full_data = pd.concat(concat_dfs)

# properly position and clean raw data, after taking adjusted close only
full_data.columns = ['date', 'ticker', 'price']
full_data.set_index('date', inplace=True)
# convert yahoo finance tickers back to numerai tickers
full_data['numerai_ticker'] = full_data.ticker.map(
    dict(zip(ticker_map['yahoo'], numerai_tickers)))
print('Data downloaded.')
print(f"Number of tickers with data: {len(full_data.numerai_ticker.unique())}")

ticker_groups = full_data.groupby('ticker')
full_data['RSI'] = ticker_groups['price'].transform(lambda x: RSI(x))

# group by era (date) and create quintile labels within each era, useful for learning relative ranking
date_groups = full_data.groupby(full_data.index)
full_data['RSI_quintile'] = date_groups['RSI'].transform(
    lambda group: pd.qcut(group, 5, labels=False, duplicates='drop'))
full_data.dropna(inplace=True)

# create lagged features grouped by ticker
ticker_groups = full_data.groupby('ticker')
num_days = 5
# lag 0 is that day's value, lag 1 is yesterday's value, etc
for day in range(num_days + 1):
    full_data[f'RSI_quintile_lag_{day}'] = ticker_groups[
        'RSI_quintile'].transform(lambda group: group.shift(day))

# create difference of the lagged features and absolute difference of the lagged features (change in RSI quintile by day)
for day in range(num_days):
    full_data[f'RSI_diff_{day}'] = full_data[
                                        f'RSI_quintile_lag_{day}'] - full_data[
                                        f'RSI_quintile_lag_{day + 1}']
    full_data[f'RSI_abs_diff_{day}'] = np.abs(
        full_data[f'RSI_quintile_lag_{day}'] -
        full_data[f'RSI_quintile_lag_{day + 1}'])

# define column names of features, target, and prediction
feature_names = [f'RSI_quintile_lag_{num}' for num in range(num_days)] + [
    f'RSI_diff_{num}' for num in range(num_days)
] + [f'RSI_abs_diff_{num}' for num in range(num_days)]
print(f'Features for training:\n {feature_names}')

Number of eligible tickers: 5394
Number of tickers in map: 5394
Number of eligible, mapped tickers: 5343


2021-06-19 22:28:16,117 INFO numexpr.utils: NumExpr defaulting to 2 threads.


[*********************100%***********************]  1000 of 1000 completed
[                       1%                       ]  14 of 1000 completed

In [ ]:



TARGET_NAME = 'target'
PREDICTION_NAME = 'signal'

# read in Signals targets
try:
    targets = pd.read_csv('historical_targets.csv')
except FileNotFoundError:
    napi.download_validation_data(dest_filename='historical_targets.csv')
    targets = pd.read_csv('historical_targets.csv')
targets['date'] = pd.to_datetime(targets['friday_date'], format='%Y%m%d')

targets.rename(columns={"bloomberg_ticker": "numerai_ticker"}, inplace=True)

# merge our feature data with Numerai targets
ML_data = pd.merge(full_data.reset_index(), targets,
                    on=['date', 'numerai_ticker']).set_index('date')
# print(f'Number of eras in data: {len(ML_data.index.unique())}')

# for training and testing we want clean, complete data only
ML_data.dropna(inplace=True)
ML_data = ML_data[ML_data.index.weekday ==
                    4]  # ensure we have only fridays
ML_data = ML_data[ML_data.index.value_counts() >
                    50]  # drop eras with under 50 observations per era

# train test split
train_data = ML_data[ML_data['data_type'] == 'train']
test_data = ML_data[ML_data['data_type'] == 'validation']

# train model
print("Training model...")
model = GradientBoostingRegressor(subsample=0.1)
model.fit(train_data[feature_names], train_data[TARGET_NAME])
print("Model trained.")

# predict test data
test_data[PREDICTION_NAME] = model.predict(test_data[feature_names])

# predict live data
# choose data as of most recent friday
last_friday = datetime.now() + relativedelta(weekday=FR(-1))
date_string = last_friday.strftime('%Y-%m-%d')

try:
    live_data = full_data.loc[date_string].copy()
except KeyError as e:
    print(f"No ticker on {e}")
    live_data = full_data.iloc[:0].copy()
live_data.dropna(subset=feature_names, inplace=True)

# get data from the day before, for markets that were closed
# on the most recent friday
last_thursday = last_friday - timedelta(days=1)
thursday_date_string = last_thursday.strftime('%Y-%m-%d')
thursday_data = full_data.loc[thursday_date_string]
# Only select tickers than aren't already present in live_data
thursday_data = thursday_data[~thursday_data.ticker.isin(
                                live_data.ticker.values)].copy()
thursday_data.dropna(subset=feature_names, inplace=True)

live_data = pd.concat([live_data, thursday_data])

print(f"Number of live tickers to submit: {len(live_data)}")
live_data[PREDICTION_NAME] = model.predict(live_data[feature_names])

# prepare and writeout example file
diagnostic_df = pd.concat([test_data, live_data])
diagnostic_df['friday_date'] = diagnostic_df.friday_date.fillna(
    last_friday.strftime('%Y%m%d')).astype(int)
diagnostic_df['data_type'] = diagnostic_df.data_type.fillna('live')
diagnostic_df[['numerai_ticker', 'friday_date', 'data_type',
                'signal']].reset_index(drop=True).to_csv(
    'example_signal_upload.csv', index=False)
print(
    'Example submission completed. Upload to signals.numer.ai for scores and live submission'
)

## My Code

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Set sensible defaults
sns.set()
sns.set_style("ticks")
sns.set_context('paper')

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 13.7 ms (started: 2021-05-15 13:28:12 +00:00)


# Loading data

In [ ]:
df_universe = pd.read_csv('https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/universe/latest.csv')
df_universe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5428 entries, 0 to 5427
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   bloomberg_ticker  5428 non-null   object
dtypes: object(1)
memory usage: 42.5+ KB
time: 487 ms (started: 2021-05-15 13:28:12 +00:00)


In [ ]:
df_universe.head(5)

,bloomberg_ticker
0,SVW AU
1,GEM AU
2,VOC AU
3,AZJ AU
4,NXT AU


time: 15.8 ms (started: 2021-05-15 13:28:12 +00:00)


In [ ]:
df_hist = pd.read_csv('https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_train_val_bbg.csv')
df_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4326873 entries, 0 to 4326872
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   bloomberg_ticker  object 
 1   friday_date       int64  
 2   data_type         object 
 3   target            float64
dtypes: float64(1), int64(1), object(2)
memory usage: 132.0+ MB
time: 6.27 s (started: 2021-05-15 14:07:28 +00:00)


In [ ]:
df_hist.sample(5)

,bloomberg_ticker,friday_date,data_type,target
77034,IDPH US,20030822,train,0.50
2407671,MM US,20140124,validation,0.25
2991896,095720 KS,20160506,validation,0.75
1228390,VMS SP,20081226,train,0.25
619079,SOV US,20060602,train,0.50


time: 207 ms (started: 2021-05-15 14:07:34 +00:00)


In [ ]:
df_hist[df_hist.bloomberg_ticker.str.startswith('AAPL')]

,bloomberg_ticker,friday_date,data_type,target
438,AAPL US,20030131,train,0.50
2813,AAPL US,20030207,train,0.50
5201,AAPL US,20030214,train,0.50
7588,AAPL US,20030221,train,0.25
9930,AAPL US,20030228,train,0.50
...,...,...,...,...
4300994,AAPL US,20210402,validation,0.75
4306419,AAPL US,20210409,validation,0.50
4311854,AAPL US,20210416,validation,0.50
4317282,AAPL US,20210423,validation,0.50


time: 1.57 s (started: 2021-05-15 14:07:35 +00:00)


In [ ]:
df_hist[df_hist.bloomberg_ticker.str.endswith('US')]

,bloomberg_ticker,friday_date,data_type,target
46,106760Q US,20030131,train,0.25
49,1108Q US,20030131,train,0.50
63,1482276D US,20030131,train,0.75
102,2386 US,20030131,train,0.75
422,9898989D US,20030131,train,0.50
...,...,...,...,...
4326866,ZS US,20210430,validation,0.50
4326867,ZTS US,20210430,validation,0.50
4326868,ZUMZ US,20210430,validation,0.50
4326869,ZUO US,20210430,validation,0.50


time: 1.46 s (started: 2021-05-15 14:07:36 +00:00)


In [ ]:
from datetime import datetime
df_hist['prefix'] = df_hist.bloomberg_ticker.apply(lambda s: str(s).split()[0])
df_hist['suffix'] = df_hist.bloomberg_ticker.apply(lambda s: str(s).split()[-1])
df_hist.sample(10)

,bloomberg_ticker,friday_date,data_type,target,prefix,suffix
1806963,TALK LN,20110715,train,0.25,TALK,LN
1164084,G CN,20080912,train,1.00,G,CN
367458,SGRE SM,20050325,train,0.50,SGRE,SM
3123587,CACI US,20161104,validation,1.00,CACI,US
4255078,NJR US,20210129,validation,0.25,NJR,US
2202032,OFIX US,20130322,validation,0.25,OFIX,US
4013545,SSE LN,20200306,validation,0.75,SSE,LN
1592088,7267 JP,20100827,train,0.25,7267,JP
3583946,MIL PW,20180720,validation,0.75,MIL,PW
2501154,METTK GA,20140606,validation,0.50,METTK,GA


time: 5.36 s (started: 2021-05-15 14:07:38 +00:00)


In [ ]:
# Sub-select only some suffixes
NB_SUFFIXES = list(df_hist.suffix.value_counts(dropna=False).head(10).index)
print(NB_SUFFIXES)
df_hist = df_hist[df_hist.suffix.isin(NB_SUFFIXES)]
print(df_hist.shape)

['US', 'JP', 'LN', 'CN', 'KS', 'TT', 'AU', 'FP', 'GR', 'HK']
(3561638, 6)
time: 902 ms (started: 2021-05-15 14:07:43 +00:00)


In [ ]:
exchange_dict = {
    'US': 'USA',
    'JP': 'JPN',
    'LN': 'GBR',
    'CN': 'CAN',
    'KS': 'PRK',
    'TT': 'TWN',
    'AU': 'AUS',
    'FP': 'FRA',
    'GR': 'DEU',
    'HK': 'CHN'
}

time: 2.83 ms (started: 2021-05-15 14:07:44 +00:00)


In [ ]:
df_hist['country_3l'] = df_hist.suffix.apply(lambda c: str(exchange_dict[c]))
df_hist['date'] = pd.to_datetime(df_hist.friday_date.apply(lambda i: datetime.strptime(str(i), "%Y%m%d")))

df_hist.sample(10)

,bloomberg_ticker,friday_date,data_type,target,prefix,suffix,country_3l,date
2840068,2891 TT,20151002,validation,0.50,2891,TT,TWN,2015-10-02
1434606,20 HK,20091218,train,0.25,20,HK,CHN,2009-12-18
3214841,DCI US,20170310,validation,0.50,DCI,US,USA,2017-03-10
241142,8184 JP,20040730,train,0.50,8184,JP,JPN,2004-07-30
324961,PEAK US,20050107,train,0.50,PEAK,US,USA,2005-01-07
3264070,002550 KS,20170519,validation,0.50,002550,KS,PRK,2017-05-19
1049051,INFA US,20080321,train,0.50,INFA,US,USA,2008-03-21
380185,5411 JP,20050422,train,0.75,5411,JP,JPN,2005-04-22
49933,AVCT US,20030620,train,0.25,AVCT,US,USA,2003-06-20
221276,INFS US,20040618,train,1.00,INFS,US,USA,2004-06-18


time: 35.8 s (started: 2021-05-15 14:07:44 +00:00)


In [ ]:
df_hist.date.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                 3561638
unique                    953
top       2021-03-26 00:00:00
freq                     4419
first     2003-01-31 00:00:00
last      2021-04-30 00:00:00
Name: date, dtype: object

time: 73.4 ms (started: 2021-05-15 14:08:20 +00:00)


In [ ]:
df_hist.data_type.value_counts()

train         1792036
validation    1769602
Name: data_type, dtype: int64

time: 305 ms (started: 2021-05-15 14:08:20 +00:00)


In [ ]:
df_universe.bloomberg_ticker.nunique(), df_hist.bloomberg_ticker.nunique()

(5428, 13107)

time: 298 ms (started: 2021-05-15 12:06:17 +00:00)


In [ ]:
import pycountry

df_lookup = pd.DataFrame(
    investpy.stocks.get_stocks_dict(
        country=None,
        columns=["symbol", "country", "name", "full_name", "isin", "currency"],
        as_json=False,
    )
)
df_lookup.sample(10)


,symbol,country,name,full_name,isin,currency
29958,SKT,new zealand,Sky Network Television,SKY Network Television Ltd,NZSKTE0001S6,NZD
39623,RFGJ,south africa,Rhodes Food Group Holdings Ltd,Rhodes Food Group Holdings Ltd,ZAE000191979,ZAR
222,TASA4,brazil,Taurus Armas PN,Taurus Armas PN,BRTASAACNPR4,BRL
34822,3701,taiwan,FIC Global,FIC Global Inc,TW0003701009,TWD
3756,CERN,united states,Cerner,Cerner Corp,US1567821046,USD
28122,3936,japan,Globalway,Globalway Inc,JP3274310006,JPY
32269,227840,south korea,Hyundai C F Inc,Hyundai C&F Inc,KR7227840006,KRW
8442,GINX,bosnia,Sindikat Metalaca BIH,Sindikat Metalaca BIH Sarajevo Sindikalna Orga...,BAGINXR00000,BAM
3366,KLAC,united states,KLA-Tencor,KLA-Tencor Corporation,US4824801009,USD
16807,RDI,united kingdom,RDI Reit,RDI Reit PLC,IM00BH3JLY32,GBP


time: 547 ms (started: 2021-05-15 13:37:15 +00:00)


In [ ]:
for c3l in df_hist.country_3l.unique():
    _df = df_hist[df_hist.country_3l == c3l]
    country = pycountry.countries.get(alpha_3=c3l).name.lower().split(',')[0]
    _lookup = df_lookup[df_lookup.country == country]
    matches = 0
    for stock in _df.prefix.unique():
        if stock in _lookup.symbol.unique():
            matches += 1
    print(country, _df.shape, matches, matches/_df.bloomberg_ticker.nunique())

korea (164061, 10) 0 0.0
china (70491, 10) 0 0.0
united states (1890789, 10) 2525 0.4535656547512125
taiwan (149879, 10) 439 0.7268211920529801
japan (550033, 10) 1154 0.8716012084592145
united kingdom (231785, 10) 263 0.36629526462395545
canada (171316, 10) 281 0.5127737226277372
france (97428, 10) 56 0.23333333333333334
germany (93893, 10) 30 0.10830324909747292
australia (141963, 10) 313 0.6101364522417154
time: 8.28 s (started: 2021-05-15 14:02:38 +00:00)


In [ ]:
def get_company(row):
    c3l = str(row.country_3l)
    prefix = str(row.prefix)
    country = pycountry.countries.get(alpha_3=c3l).name.lower().split(',')[0]
    data = df_lookup[(df_lookup.country == country) & (df_lookup.symbol == prefix)][['name', 'full_name', 'currency']]
    if data.shape[0] != 1:
        return np.nan, np.nan, np.nan
    return data.name.values[0], data.full_name.values[0], data.currency.values[0]

_df_hist = df_hist.copy().sample(1000)
companies, fnames, currencies = _df_hist.apply(lambda x: get_company(x), axis=1).T
_df_hist['company'] = companies
_df_hist

In [ ]:
df_lookup[df_lookup.symbol.str.startswith('AAPL')]

,symbol,country,name,full_name,isin,currency
86,AAPL,argentina,Apple DRC,Apple Inc DRC,ARDEUT116183,ARS
421,AAPL34,brazil,Apple Inc BDR,Apple Inc BDR,BRAAPLBDR004,BRL
2743,AAPL,mexico,Apple,Apple Inc,US0378331005,MXN
3330,AAPL,united states,Apple,Apple Inc,US0378331005,USD
7940,AAPL,austria,Apple,Apple Inc,US0378331005,EUR
11529,AAPLE,italy,Apple,Apple Inc,US0378331005,EUR
14905,AAPLEUR,switzerland,Apple,Apple Inc,US0378331005,EUR
15005,AAPLUSD,switzerland,Apple,Apple Inc,US0378331005,USD
15054,AAPL,switzerland,Apple,Apple Inc,US0378331005,CHF


time: 41.3 ms (started: 2021-05-15 14:12:09 +00:00)


In [ ]:
search_results = investpy.search_quotes(
    text='apple',
    products=['stocks'],
    countries=['united states'],
    n_results=10
)

for search_result in search_results[:1]:
   print(search_result)
   search_result.retrieve_historical_data(from_date='01/01/2019', to_date='01/01/2020')
   print(search_result.data.head())

{"id_": 6408, "name": "Apple Inc", "symbol": "AAPL", "country": "united states", "tag": "/equities/apple-computer-inc", "pair_type": "stocks", "exchange": "NASDAQ"}
             Open   High    Low  Close     Volume
Date                                             
2019-01-02  38.72  39.71  38.56  39.48  148159008
2019-01-03  35.99  36.43  35.50  35.55  365248992
2019-01-04  36.13  37.14  35.95  37.06  234428000
2019-01-07  37.17  37.21  36.47  36.98  219111008
2019-01-08  37.39  37.96  37.13  37.69  164100992
time: 709 ms (started: 2021-05-15 12:26:11 +00:00)


In [ ]:
matches = 0
for ticker in tqdm(df_hist.bloomberg_ticker.unique()):
    if ticker in df_lookup['isin']:
        matchers += 1
print(matches, matches/df_hist.bloomberg_ticker.nunique())

100%|██████████| 13107/13107 [00:00<00:00, 144238.04it/s]


0 0.0
time: 698 ms (started: 2021-05-15 12:21:26 +00:00)


In [ ]:
# https://github.com/ranaroussi/yfinance
import yfinance as yf

yf.download(
    tickers=['AAPL', 'MSFT', 'APBR', 'PROL'],
    period='1mo',
    interval='1m',
    group_by='ticker',
)

[*********************100%***********************]  4 of 4 completed

4 Failed downloads:
- PROL: No data found, symbol may be delisted
- MSFT: 1m data not available for startTime=1618402682 and endTime=1621081082. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
- APBR: No data found, symbol may be delisted
- AAPL: 1m data not available for startTime=1618402682 and endTime=1621081082. Only 7 days worth of 1m granularity data are allowed to be fetched per request.


Empty DataFrame
Columns: [(PROL, Open), (PROL, High), (PROL, Low), (PROL, Close), (PROL, Adj Close), (PROL, Volume), (MSFT, Open), (MSFT, High), (MSFT, Low), (MSFT, Close), (MSFT, Adj Close), (MSFT, Volume), (APBR, Open), (APBR, High), (APBR, Low), (APBR, Close), (APBR, Adj Close), (APBR, Volume), (AAPL, Open), (AAPL, High), (AAPL, Low), (AAPL, Close), (AAPL, Adj Close), (AAPL, Volume)]
Index: []

time: 233 ms (started: 2021-05-15 12:18:02 +00:00)
